In [18]:
!rm question.sqlite

import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base 
from sqlalchemy import Column, Integer, String, Float, Text, Boolean
from sqlalchemy.orm import Session


import pymysql
pymysql.install_as_MySQLdb()

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
file = './db.csv'
db_df = pd.read_csv(file)

In [20]:
db_df.columns.tolist()

['id',
 'topic',
 'date',
 'qNum',
 'qStatement',
 'a1',
 'a2',
 'a3',
 'a4',
 'a5',
 'pic']

In [21]:
db_df.head()

,id,topic,date,qNum,qStatement,a1,a2,a3,a4,a5,pic
0,Apr.2015.1,Absolute Values,April.2015,1,1. |3(-2) + 4| = ?,-2,2,5,9,10,NaN
1,Dec.2015.40,Absolute Values,December.2015,40,40. What is the set of real solutions for |x|2...,{2},"{-2, 2}","{-1, 2}","{1, 2}","{-2, -1, 1, 2}",NaN
2,Jun.2016.15,Absolute Values,June.2016,15,15. |6(-7) + 4(8)| = ?,-144,-10,10,74,144,NaN
3,Jun.2016.54,Absolute Values,June.2016,54,54. The graphs of the functions y = f(x) = (x ...,|f(x)| = f(x),|g(x)| = g(x),f(3) = g(3),f(3 ½ ) = g(3 ½ ),f(g(1)) = 6,*******picture******
4,Apr.2016.1,Absolute Values,April.2016,1,1. What is |5 - x| when x=9?,-14,-4,4,9,14,NaN


In [22]:
engine = create_engine("sqlite:///db.sqlite")
conn = engine.connect()
Base = declarative_base()

In [23]:
class Questions(Base):
    __tablename__ = "math"
    id = Column(String(255), primary_key = True)
    topic = Column(String(255))
    date = Column(String(50))
    qNum = Column(Integer)
    qStatement = Column(String(1000))
    a1 = Column(String(255))
    a2 = Column(String(255))
    a3 = Column(String(255))
    a4 = Column(String(255))
    a5 = Column(String(255))
    pic = Column(String(50))           

In [24]:
db_dict = db_df.to_dict(orient='records')

In [25]:
Base.metadata.create_all(engine)
metadata = sqlalchemy.schema.MetaData(bind=engine,reflect=True)

table = sqlalchemy.Table('math', metadata, autoload=True)
conn.execute(table.delete())
conn.execute(table.insert(), db_dict)


C:\Users\Bryant\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  from ipykernel import kernelapp as app


In [26]:
conn.execute("SELECT * FROM math LIMIT 2").fetchall()

[('Apr.2015.1', 'Absolute Values', 'April.2015', '1', '1. |3(-2) + 4| = ?', '-2', '2', '5', '9', '10', None),
 ('Dec.2015.40', 'Absolute Values', 'December.2015', '40', '40. What is the set of real solutions for |x|2 - |x| - 2 = 0?', '{2}', '{-2, 2}', '{-1, 2}', '{1, 2}', '{-2, -1, 1, 2}', None)]

In [29]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker, Query
import json

engine = create_engine('sqlite:///data/mathTest.sqlite', convert_unicode=True, echo=False)
Base = declarative_base()
Base.metadata.reflect(engine)


class Questions(Base):
    __table__ = Base.metadata.tables['math']


session = scoped_session(sessionmaker(bind=engine))


def questions(input_data):

    questionReturn = session.query(Questions.qStatement,
                                   Questions.a1,
                                   Questions.a2,
                                   Questions.a3,
                                   Questions.a4,
                                   Questions.a5).filter(Questions.topic == input_data)
    
    dataDict = [{'statement': each[0],
                 'a1': each[1],
                 'a2': each[2],
                 'a3': each[3],
                 'a4': each[4],
                 'a5': each[5]
                 } for each in questionReturn]

    return dataDict

OperationalError: (sqlite3.OperationalError) unable to open database file (Background on this error at: http://sqlalche.me/e/e3q8)

In [113]:
questions('April_2015_1')[0]['Question']

'The circle in the standard (x,y) coordinate plane below has center (-8.5, 7.5) and has radius 5 coordinate units.'